In [ ]:
import ipywidgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg, Heartbeat
from IPython.display import display

# Controller setup (adjust index if needed) 
controller = widgets.Controller(index=1)
display(controller)

# Robot setup 
robot = Robot()
left_link  = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

# Camera resolution 
cam_width = 320
cam_height = 240

# Camera setup 
camera = Camera.instance(width=cam_width, height=cam_height)
image = widgets.Image(format='jpeg', width=cam_width, height=cam_height)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

# Safety: heartbeat stop 
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
heartbeat.observe(handle_heartbeat_status, names='status')
